In [10]:
# Import libs
import geemap, ee
import geemap.colormaps as cm
# initialize our map
Map = geemap.Map()

In [11]:
# get our Nepal boundary
# I have taken level 0 data for country data from FAO datasets
aoi = ee.FeatureCollection("FAO/GAUL/2015/level0")\
   .filter(ee.Filter.eq('ADM0_NAME','Nepal')).geometry() # adjust indentation here, May get error

In [12]:
def getEVI(image):
    # Compute the EVI using an expression.
    EVI = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8').divide(10000),
            'RED': image.select('B4').divide(10000),
            'BLUE': image.select('B2').divide(10000)
        }).rename("EVI")

    image = image.addBands(EVI)

    return(image)

In [13]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [14]:
# filter sentinel 2 images and apply the EVI formula, finally we obtain 
# single image with median operation

Sentinel_data = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate("2022-03-01","2022-03-31").filterBounds(aoi) \
    .map(getEVI).map(addDate).median()

In [15]:
# set some thresholds
#  A nice EVI palette
palett = [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
    '74A901', '66A000', '529400', '3E8601', '207401', '056201',
    '004C00', '023B01', '012E01', '011D01', '011301']

pall = {"min":0.1, "max":0.8, 'palette':palett}



Map.centerObject(aoi, 8)
Map.addLayer(Sentinel_data.clip(aoi).select('EVI'), pall, "EVI")

Map.addLayerControl()
Map = geemap.Map()
